In [9]:
{"Hello": [5,[3,10,23,27]]}

{'Hello': [5, [3, 10, 23, 27]]}

### Exercise 1: What does the following representation mean? {23: [3, [0, 12, 28]]}

#### 23 is posting list
#### 3 is frequency at level positinal index
#### another numbers are level positinal index

# Steps to build a Positional Index

In [5]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\RaKaN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\RaKaN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import os
import string
import copy
import pickle 

In [31]:
title = "comp.graphics"
os.chdir("D:/mini_newsgroups")
paths= []
for (dirpath, dirnames, filenames) in os.walk(str(os.getcwd())+'/'+title+'/'): 
    for i in filenames: 
        paths.append(str(dirpath)+str("\\")+i)
print(dirpath)


D:\mini_newsgroups/comp.graphics/


In [32]:
paths[0]

'D:\\mini_newsgroups/comp.graphics/\\37916'

In [33]:
len(paths)

100

## Preprocessing

### Exercise 2: Write a set of functions that is able to remove:
### stop words, punctuation, apostrophe, single characters, and
### header. In addition, you have to convert words into lowercase,
### stem the inter words and convert numbers to its equivalent words. 

## Removing stop words

In [13]:
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words:
            new_text = new_text + " " + w
    return np.char.strip(new_text)

##  Removing punctuation 

In [14]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], '')
        data = np.char.replace(data, " ", " ")
    data = np.char.replace(data, ',', '')
    return data

## Convert to lowercase 

In [15]:
def convert_lower_case(data):
    return np.char.lower(data) 

## Stemming

In [16]:
def stemming(data):
    stemmer= PorterStemmer()
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return np.char.strip(new_text)

## Converting numbers to its equivalent words

In [34]:
def convert_numbers(data):
    data = np.char.replace(data, "0", " zero ")
    data = np.char.replace(data, "1", " one ")
    data = np.char.replace(data, "2", " two ")
    data = np.char.replace(data, "3", " three ")
    data = np.char.replace(data, "4", " four ")
    data = np.char.replace(data, "5", " five ")
    data = np.char.replace(data, "6", " six ")
    data = np.char.replace(data, "7", " seven ")
    data = np.char.replace(data, "8", " eight ")
    data = np.char.replace(data, "9", " nine ")
    return data

## Removing header 

In [36]:
def remove_header(data):
    try:
        ind = data.index('\n\n')
        data = data[ind:]
    except:
        print("No Header")
    return data

## Removing apostrophe

In [37]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

## Removing single characters

In [38]:
def remove_single_characters(data):
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if len(w) > 1:
            new_text = new_text + " " + w
    return np.char.strip(new_text)

# Generating Postings 

In [40]:
def preprocess(data, query):
    if not query:
        data = remove_header(data) 
        data = convert_lower_case(data)
        data = convert_numbers(data)
        data = remove_punctuation(data)
        data = remove_stop_words(data)
        data = remove_apostrophe(data)
        data = remove_single_characters(data)
        data = stemming(data) 
    return data

In [49]:
postings = pd.DataFrame()
frequency = pd.DataFrame()
doc = 0
for path in paths:
    file = open(path, 'r', encoding='cp1250')
    text = file.read().strip()
    file.close()
    preprocessed_text = preprocess(text, False)
    if doc%100 == 0:
        print(doc)
    tokens = word_tokenize(str(preprocessed_text))
    
    pos = 0
    for token in tokens:
        if token in postings:
            p = postings[token][0]
            
            k = [a[0] for a in p]
            if doc in k:
                for a in p:
                    if a[0] == doc:
                        a[1].add(pos)
            else:
                p.append([doc,{pos}])
                frequency[token][0] += 1
        else:
            postings.insert(value=[[[doc, {pos}]]], loc=0, column=token)
            frequency.insert(value=[1], loc=0, column=token)
            
        pos += 1
    doc += 1

0


In [51]:
postings

,moneydav,wastedyour,stuffif,programto,fontmong,printerhowev,lazer,reallyfor,rippingalthough,documentto,...,simpl,sipp,call,routin,render,librari,describ,file,got,recent
0,"[[99, {106}]]","[[99, {105}]]","[[99, {99}]]","[[99, {94}]]","[[99, {92}]]","[[99, {88}]]","[[99, {87}]]","[[99, {84}]]","[[99, {77}]]","[[99, {73}]]",...,"[[0, {9}], [1, {48}], [17, {952, 4329, 4531, 4...","[[0, {8, 29}]]","[[0, {7}], [5, {39}], [10, {72}], [17, {101, 2...","[[0, {33, 6}], [5, {114, 123}], [10, {12}], [1...","[[0, {32, 5}], [1, {26}], [17, {761}], [46, {1...","[[0, {4, 31}], [1, {16}], [17, {1088, 3975, 43...","[[0, {3}], [17, {1183}], [65, {41}], [76, {24,...","[[0, {2}], [5, {154}], [7, {17, 30}], [8, {32,...","[[0, {1}], [29, {49}], [33, {53}], [37, {111}]...","[[0, {0}], [5, {153}], [13, {0}], [17, {5035}]..."


In [53]:
postings["call"]

0    [[0, {7}], [5, {39}], [10, {72}], [17, {101, 2...
Name: call, dtype: object

## Save outputs

In [56]:
postings.to_pickle(title + "_positional_postings") 
frequency.to_pickle(title + "_positional_frequency") 

# Search Query

## Read the stored posting list

In [58]:
postings = pd.read_pickle(title + "_positional_postings") 

## Read frequency of a term as follows

In [59]:
frequency = pd.read_pickle(title + "_positional_frequency") 

## Exercise3

In [60]:
#### 

In [66]:
def get_word_postings(word):
    preprocessed_word = str(preprocess(word, True))
    print(preprocessed_word)
    print("Frequency:",frequency[preprocessed_word][0])
    print("Postings List:",postings[preprocessed_word][0])
    
def get_positions(posting_values, doc):
    for posting_value in posting_values:
        if posting_value[0] == doc:
            return posting_value[1]
    return {}

def gen_init_set_matchings(word):
    init = []
    word_postings = postings[word][0]
    for word_posting in word_postings:
        for positions in word_posting[1]:
            init.append((word_posting[0], positions))
    return init

def match_positional_index(init, b):
    matched_docs = []
    for p in init:
        doc = p[0]
        pos = p[1]

        count = 0

        for k in b:
            pos = pos+1
            k_pos = postings[k][0]
            docs_list = [z[0] for z in k_pos]
            if doc in docs_list:
                doc_positions = get_positions(k_pos, doc)
                if pos in doc_positions:
                    count += 1
                else:
                    count += 1
                    break

            if count == len(b):
                matched_docs.append(p[0])
    return set(matched_docs)

def run_query(query):
    processed_query = preprocess(query, True)
    print(processed_query)

    query_tokens = word_tokenize(str(processed_query))
    print(query_tokens)
    
    if len(query_tokens)==1:
        print("Total Document Mathces", [a[0] for a in postings[query][0]])
        return [a[0] for a in postings[query][0]]
    
    init_word = query_tokens[0]
    init_matches = gen_init_set_matchings(init_word)

    query_tokens.pop(0)
    total_matched_docs = match_positional_index(init_matches, query_tokens)
    print("Total Document Matches:", total_matched_docs)
    return total_matched_docs

In [81]:
get_word_postings("fontmong")

fontmong
Frequency: 1
Postings List: [[99, {92}]]


In [91]:
query = "fontmong got"
lists = run_query(query)

fontmong got
['fontmong', 'got']
Total Document Matches: set()


In [84]:
postings["render"]

0    [[0, {32, 5}], [1, {26}], [17, {761}], [46, {1...
Name: render, dtype: object

## Home Task
### Exercise5

In [90]:
query = postings
lists = run_query(query)

        moneydav     wastedyour       stuffif     programto      fontmong  \
0  [[99, {106}]]  [[99, {105}]]  [[99, {99}]]  [[99, {94}]]  [[99, {92}]]   

   printerhowev         lazer     reallyfor rippingalthough    documentto  \
0  [[99, {88}]]  [[99, {87}]]  [[99, {84}]]    [[99, {77}]]  [[99, {73}]]   

   ...                                              simpl            sipp  \
0  ...  [[0, {9}], [1, {48}], [17, {952, 4329, 4531, 4...  [[0, {8, 29}]]   

                                                call  \
0  [[0, {7}], [5, {39}], [10, {72}], [17, {101, 2...   

                                              routin  \
0  [[0, {33, 6}], [5, {114, 123}], [10, {12}], [1...   

                                              render  \
0  [[0, {32, 5}], [1, {26}], [17, {761}], [46, {1...   

                                             librari  \
0  [[0, {4, 31}], [1, {16}], [17, {1088, 3975, 43...   

                                             describ  \
0  [[0, {3}], [17, {1183}],